## Imports

In [234]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from utils import rgb2gray
from hog import hog
from SVM import OVO_test, OVO_train
from joblib import dump, load

In [246]:
Xtr = pd.read_csv('data/Xtr.csv',header=None,sep=',',usecols=range(3072))
Xte = pd.read_csv('data/Xte.csv',header=None,sep=',',usecols=range(3072))
Ytr_ = pd.read_csv('data/Ytr.csv')#, index_col = None, header = None)

## Data preprocessing

In [247]:
Xtr = np.array(Xtr).reshape(5000, 3,32, 32)#.swapaxes(1,2).swapaxes(2,3)
Xte = np.array(Xte).reshape(2000, 3,32, 32).swapaxes(1,2).swapaxes(2,3)
Ytr = np.array(Ytr_['Prediction'])

In [248]:
from torchvision.transforms import RandomCrop, Resize, RandomHorizontalFlip, RandomVerticalFlip, ColorJitter, GaussianBlur, RandomPerspective
import torch

## Hog processing

In [249]:
from skimage.feature import hog as hog2
#Xtr_hog = np.array([hog(image) for image in Xtr])
#Xte_hog = np.array([hog(image) for image in Xte])

## SVM

In [250]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [251]:
X_train, X_test, y_train, y_test = train_test_split(Xtr, Ytr, test_size=0.2, random_state=0)

In [252]:
Xtr = torch.Tensor(X_train)
Xtr_randomcrop24 = Resize(32)(RandomCrop(24)(Xtr))
Xtr_RandomHorizontalFlip = RandomHorizontalFlip()(Xtr)
Xtr_RandomVerticalFlip = RandomVerticalFlip()(Xtr)
X_tr_ColorJitter = ColorJitter(brightness=.5, hue=.3)(Xtr)
X_tr_GaussianBlur = GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))(Xtr)
x_tr_RandomPerspective = RandomPerspective(distortion_scale=0.6, p=1.0)(Xtr)

In [253]:
Xtr = np.array(Xtr).swapaxes(1,2).swapaxes(2,3)
Xtr_randomcrop24 = np.array(Xtr_randomcrop24).swapaxes(1,2).swapaxes(2,3)
Xtr_RandomHorizontalFlip = np.array(Xtr_RandomHorizontalFlip).swapaxes(1,2).swapaxes(2,3)
Xtr_RandomVerticalFlip = np.array(Xtr_RandomVerticalFlip).swapaxes(1,2).swapaxes(2,3)
X_tr_ColorJitter = np.array(X_tr_ColorJitter).swapaxes(1,2).swapaxes(2,3)
X_tr_GaussianBlur = np.array(X_tr_GaussianBlur).swapaxes(1,2).swapaxes(2,3)
x_tr_RandomPerspective = np.array(x_tr_RandomPerspective).swapaxes(1,2).swapaxes(2,3)

In [254]:
X_train = np.vstack((Xtr, Xtr_randomcrop24, Xtr_RandomHorizontalFlip, Xtr_RandomVerticalFlip, X_tr_ColorJitter, X_tr_GaussianBlur, x_tr_RandomPerspective))
y_train = np.hstack((y_train, y_train, y_train,y_train,y_train,y_train, y_train))

In [255]:
X_test = X_test.swapaxes(1,2).swapaxes(2,3)
X_train = np.array([hog2(image,orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3),visualize=False, channel_axis=-1) for image in X_train])
X_test = np.array([hog2(image,orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3),visualize=False, channel_axis=-1) for image in X_test])

In [256]:
clf = SVC(C=1)
clf.fit(X_train, y_train)

SVC(C=1)

In [257]:
y_pred = clf.predict(X_test)

In [258]:
clf.score(X_test, y_test)

0.515